In [1]:
import nltk
#nltk.download() #check that everything is installed on the PC and uptodate

In [2]:
# toggles
# stack_to_save is for saving the stack classifier once it is run
# stack_to_load is for using a prior saved stack classifier instead of training it again

stack_to_save = 1 # must be 0 or 1
stack_to_load = 0 # must be 0 or 1, cant be same as stack_to_save

# if both = 0 then the trained models will not be saved and no model will be loaded in
# i.e. the program will just be run with nothing saved


In [3]:
# pandas dataframes to hold the tweets
import pandas as pd
import numpy as np
from IPython.display import display

%matplotlib inline
pd.options.display.max_columns = None

# use nltk for the natural language processing
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# pickle to save the trained models to reload in prediction program
# csv to save the commmon words to reload in prediction program
import pickle
import csv

# Machine Learning
from sklearn.model_selection import train_test_split
from mlxtend.classifier import StackingCVClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import metrics
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier


In [4]:
# use random_state of 0 to stay consistent when training models are run
# and therefore able to detect changes in the output
random_state = 0

In [5]:
# check on the panda version and its dependencies
# i run this from time to time to ensure all is up to date
pd.__version__
#pd.show_versions()

'0.25.3'

In [6]:
# upload the training data into a dataframe, kaggle https://www.kaggle.com/kazanova/sentiment140
# add the column names
df_train = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding = "ISO-8859-1",header=None)
df_train.columns = ["target", "id", "date", "flag", "user", "text"]

In [7]:
#
# exploring the training data
#

In [8]:
df_train.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [9]:
# there is no missing data
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
target    1600000 non-null int64
id        1600000 non-null int64
date      1600000 non-null object
flag      1600000 non-null object
user      1600000 non-null object
text      1600000 non-null object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [10]:
# target: 0 = negative, 4 = positive
df_train['target'].unique()

array([0, 4], dtype=int64)

In [11]:
# seems there is exactly 800k positive and 800k negative tweets, total tweets 1.6m
df_train['target'].value_counts()

4    800000
0    800000
Name: target, dtype: int64

In [12]:
#
# set up training for bag of words as a first try
#

In [13]:
# as a first step, i will need to extract features using the nltk toolset
# step 1, clean each tweet to get rid of labels, @names, urls and put into lower case
# step 2, only look at the adjectives and adverbs for the sentiment
# Step 3 filter out the stopwords and stem the rest
# step 4, work out which are the most popular words and use them as features
# step 5, add the features to the dataframe and populate each tweet with True or False for each feature
# Step 6, Train the model


In [14]:
with pd.option_context('display.max_colwidth', 140):
    print(df_train['text'][0:5])

0    @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
1        is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!
2                              @Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds
3                                                                        my whole body feels itchy and like its on fire 
4        @nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. 
Name: text, dtype: object


In [15]:
# step 1

text_cleaned = []

for i in range(len(df_train.index)):
    tweet_to_clean = df_train['text'][i]
    tweet_to_clean = re.sub(r'#([^\s]+)', r'\1', tweet_to_clean) # all #hashtag goes to hashtag
    tweet_to_clean = re.sub('@[^\s]+','', tweet_to_clean) # all @name deleted
    tweet_to_clean = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet_to_clean) # all urls deleted
    tweet_to_clean = re.sub('[\s]+', ' ', tweet_to_clean) # converts all the resulting whitespace to one space only 
    tweet_to_clean = tweet_to_clean.lower() # convert all to lower case
    
    text_cleaned.append(tweet_to_clean)

df_train['text_cleaned'] = text_cleaned # new dataframe column with the cleaned text

In [16]:
with pd.option_context('display.max_colwidth', 140):
    print (df_train['text_cleaned'][0:5])


0                                     - awww, that's a bummer. you shoulda got david carr of third day to do it. ;d
1    is upset that he can't update his facebook by texting it... and might cry as a result school today also. blah!
2                                    i dived many times for the ball. managed to save 50% the rest go out of bounds
3                                                                   my whole body feels itchy and like its on fire 
4                    no, it's not behaving at all. i'm mad. why am i here? because i can't see you all over there. 
Name: text_cleaned, dtype: object


In [17]:
# i will also need to change the column headers in case they conflict with the feature columns later
# i.e. if a feature word is 'text' or 'date' or 'flag' etc
# add'__' to each column header

for i in range(6):
    df_train.rename(columns={df_train.columns[i]: df_train.columns[i]+'__'}, inplace=True)
    
df_train.head()

,target__,id__,date__,flag__,user__,text__,text_cleaned
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","- awww, that's a bummer. you shoulda got davi..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,is upset that he can't update his facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,i dived many times for the ball. managed to s...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","no, it's not behaving at all. i'm mad. why am..."


In [18]:
# step 2

#adjective_flag = ["J","R"] # looking for adjectives and adverbs only
adjective_flag = ["J","R","V","N"] # looking for adjectives, adverbs, verbs and nouns

# the trained models seem to work better when verbs and nouns included


tweets_adjectives = []

for i in range(len(df_train.index)):
    tweets_toFilter = df_train['text_cleaned'][i]
    words = word_tokenize(tweets_toFilter)
    word_type = nltk.pos_tag(words)

    new_tweet = []
    for w in word_type:
        if w[1][0] in adjective_flag:
            new_tweet.append(w[0].lower())
    tweets_adjectives.append(new_tweet)

df_train['text_adjectives'] = tweets_adjectives # new dataframe column with the filtered text

In [19]:
with pd.option_context('display.max_colwidth', 140):
    print(df_train['text_adjectives'][0:5])

0                       [awww, 's, bummer, shoulda, got, david, carr, third, day, do, d]
1    [is, upset, n't, update, facebook, texting, cry, result, school, today, also, blah]
2                      [i, dived, many, times, ball, managed, save, %, rest, go, bounds]
3                                                [whole, body, feels, itchy, like, fire]
4                       ['s, not, behaving, i, 'm, mad, am, i, here, i, n't, see, there]
Name: text_adjectives, dtype: object


In [20]:
# step 3

stop_words = stopwords.words("english")
stemmer = PorterStemmer()

In [21]:
tweets_filtStem = []

for i in range(len(df_train.index)):
    tweets_toFilter = df_train['text_adjectives'][i]
    #words = word_tokenize(tweets_toFilter) # no need to tokenize again, already done

    new_tweet = []
    for w in tweets_toFilter:
        if w not in stop_words:
            new_tweet.append(stemmer.stem(w))
    tweets_filtStem.append(new_tweet)

df_train['text_filtered_stemmed'] = tweets_filtStem # new dataframe column with the filtered text

In [22]:
with pd.option_context('display.max_colwidth', 140):
    print(df_train['text_filtered_stemmed'][0:5])

0                      [awww, 's, bummer, shoulda, got, david, carr, third, day]
1    [upset, n't, updat, facebook, text, cri, result, school, today, also, blah]
2                      [dive, mani, time, ball, manag, save, %, rest, go, bound]
3                                         [whole, bodi, feel, itchi, like, fire]
4                                                 ['s, behav, 'm, mad, n't, see]
Name: text_filtered_stemmed, dtype: object


In [23]:
# Step 4
# work out all the words used, in order of use, the most common will be the features

tweet_words=[]

for i in range(len(df_train.index)):
    for w in df_train['text_filtered_stemmed'][i]:
        tweet_words.append(w.lower())

wordDict_freq=nltk.FreqDist(tweet_words)

In [24]:
wordDict_freq_sorted = {k: v for k, v in sorted(wordDict_freq.items(), key=lambda item: item[1], reverse=True)}

In [25]:
len(wordDict_freq_sorted)

333163

In [26]:
# there will be over 30,000 words, so use the top 800 only
# that will give us 800 features

number_of_features = 800
common_words = list(wordDict_freq_sorted.keys())[:number_of_features]

In [27]:
#common_words

In [28]:
# save common_words (the features) to a csv file to use in the predictions sentiment analysis code
# this will overwrite the current file, so if training from new tweets need to save a new model also

with open('common_words.csv','w') as output_file:
    csv_out = csv.writer(output_file)
    csv_out.writerow(common_words)

In [29]:
# Step 5
# and now for each tweet i can assign a list of x features
# which is each of the common_words marked 1 (present) or 0 (not present)

allTweet_features = []
for i in range(len(df_train.index)):    
    tweet_features = []
    words=set(df_train['text_filtered_stemmed'][i])
    for common_word in common_words:
        if common_word in words:
            tweet_features.append(1)
        else:
            tweet_features.append(0)
    allTweet_features.append(tweet_features)


In [30]:
# from these lists build them into the columns in the dataframe
# for each common word

number_of_tweets = len(df_train.index)

for i in range(number_of_features):
    dataframe_feature_data = []
    for ii in range(number_of_tweets):
        dataframe_feature_data.append(allTweet_features[ii][i])
    df_train[common_words[i]] = dataframe_feature_data    


In [31]:
df_train.head()

,target__,id__,date__,flag__,user__,text__,text_cleaned,text_adjectives,text_filtered_stemmed,n't,go,'m,'s,get,day,good,work,love,quot,got,today,time,thank,want,back,miss,lol,know,u,see,feel,think,realli,im,amp,hope,night,watch,still,make,need,new,well,home,look,come,much,last,morn,twitter,tomorrow,wish,great,wait,'re,sleep,haha,sad,fun,tri,right,week,happi,follow,bad,'ve,thing,sorri,tonight,friend,say,way,take,gon,nice,better,hate,even,bed,start,tweet,peopl,school,show,hour,guy,play,weekend,final,hey,lt,let,awesom,use,dont,never,yeah,cant,soon,next,long,littl,rain,pleas,tire,first,everyon,best,movi,year,sick,life,wan,find,girl,sure,call,suck,..,done,help,head,alway,bore,talk,keep,like,alreadi,cool,lot,someth,live,eat,phone,leav,readi,hurt,read,made,man,oh,enjoy,x,went,ok,song,hous,yet,yay,ur,sound,thought,pretti,ever,mayb,na,excit,finish,away,summer,amaz,game,guess,tell,old,mean,one,listen,someon,earli,damn,lost,left,give,check,bit,babi,big,hear,parti,hot,noth,late,end,actual,glad,birthday,omg,happen,wow,also,pic,stop,sun,later,weather,wonder,@,put,mom,stuff,ta,saw,stay,fuck,hard,said,run,ya,car,exam,god,world,yesterday,kid,meet,music,that,job,beauti,updat,sunday,ye,mani,friday,post,seem,monday,n,hi,video,found,luck,cold,move,book,die,busi,gone,cri,poor,buy,boy,anyth,least,plan,shop,famili,woke,total,studi,hair,aww,food,almost,cute,iphon,lunch,believ,far,pictur,drink,free,chang,month,place,everyth,sweet,dinner,welcom,funni,tho,win,class,gt,anyon,shit,forward,turn,till,drive,sit,mine,ask,r,okay,walk,name,idea,dream,dad,caus,stupid,write,send,clean,real,coffe,enough,hahaha,ill,room,wrong,probabl,wake,didnt,dog,saturday,anymor,fan,money,minut,person,sooo,hit,ha,rememb,tv,xx,break,aw,headach,came,whole,brother,rock,seen,face,fail,repli,train,beach,hang,eye,kill,blog,=,rest,open,kinda,crazi,pain,close,mother,took,comput,care,word,quit,hell,super,worri,news,true,abl,anyway,hello,forgot,goodnight,els,problem,trip,part,bring,heart,offic,kind,photo,pay,full,link,soo,sister,mind,danc,cuz,b,boo,alon,stuck,internet,fall,test,ticket,cours,sometim,heard,pick,ugh,email,ah,site,awww,set,learn,hehe,pass,vote,interest,text,concert,wont,shower,btw,season,dude,add,awak,fine,fix,],told,ago,hand,favorit,suppos,visit,breakfast,goe,/,cat,sunni,onlin,ice,catch,serious,broke,facebook,bought,spend,ass,pack,cut,wear,lucki,crap,half,asleep,smile,reason,june,ride,lmao,hungri,agre,definit,sign,afternoon,mad,red,instead,ladi,jealou,sore,citi,stori,w/,page,figur,second,join,messag,graduat,album,top,nap,homework,soooo,xd,la,list,congrat,date,togeth,tour,bye,coupl,laptop,dead,park,holiday,wed,sing,save,high,til,water,goin,store,award,+,revis,yea,drop,point,star,moment,complet,relax,dear,perfect,dress,download,church,ppl,short,account,share,side,cook,ipod,line,offici,hug,answer,tea,decid,realiz,youtub,ive,town,annoy,nite,forget,lose,order,scare,weird,gym,mood,unfortun,lil,cream,min,outsid,math,understand,air,english,mum,differ,fb,fli,usual,ate,less,chocol,pool,knew,episod,band,comment,worst,[,upload,fast,support,window,chanc,horribl,flight,load,kick,parent,around,london,question,broken,da,throat,cheer,laugh,team,black,chat,card,sunshin,special,xxx,%,sat,sent,upset,gave,followfriday,sleepi,number,slow,depress,beat,ahh,ad,green,colleg,yep,mac,leg,bless,record,fell,moon,project,appar,paper,what,jona,cake,tuesday,garden,beer,wors,rather,vacat,power,film,websit,doesnt,possibl,longer,warm,app,easi,bet,bodi,bday,blue,due,fair,p,mess,huge,light,c,spent,freak,shoe,finger,hmm,celebr,disappoint,safe,past,lay,juli,cancel,sim,chill,miley,mtv,shame,cousin,bu,earlier,flu,googl,burn,lazi,absolut,age,voic,thx,stomach,white,remind,nope,hold,forev,swim,bike,thursday,manag,stress,wtf,touch,camp,especi,met,shot,camera,appreci,babe,slept,current,sort,son,crash,bum,boyfriend,lie,uk,pray,david,appl,sadli,cd,idk,box,shoot,dm,fight,piss,havent,tom,ahhh,airport,co,type,cover,myspac,block,case,terribl,arriv,present,servic,meant,si,invit,note,expect,bbq,road,pizza,club,bitch,hospit,father,

In [32]:
# Step 6

# define X and y
# recall that columns have been renamed to avoid conflict with features

feature_cols = [col for col in list(df_train.columns) 
                if col not in {'id__','target__', 'date__', 'flag__', 'user__', 'text__', 'text_adjectives',
                               'text_cleaned','text_filtered_stemmed'}]
# i.e. all the feature columns only

X = df_train[feature_cols]
y = df_train['target__']

X.shape, y.shape

((1600000, 800), (1600000,))

In [33]:
X.head()

,n't,go,'m,'s,get,day,good,work,love,quot,got,today,time,thank,want,back,miss,lol,know,u,see,feel,think,realli,im,amp,hope,night,watch,still,make,need,new,well,home,look,come,much,last,morn,twitter,tomorrow,wish,great,wait,'re,sleep,haha,sad,fun,tri,right,week,happi,follow,bad,'ve,thing,sorri,tonight,friend,say,way,take,gon,nice,better,hate,even,bed,start,tweet,peopl,school,show,hour,guy,play,weekend,final,hey,lt,let,awesom,use,dont,never,yeah,cant,soon,next,long,littl,rain,pleas,tire,first,everyon,best,movi,year,sick,life,wan,find,girl,sure,call,suck,..,done,help,head,alway,bore,talk,keep,like,alreadi,cool,lot,someth,live,eat,phone,leav,readi,hurt,read,made,man,oh,enjoy,x,went,ok,song,hous,yet,yay,ur,sound,thought,pretti,ever,mayb,na,excit,finish,away,summer,amaz,game,guess,tell,old,mean,one,listen,someon,earli,damn,lost,left,give,check,bit,babi,big,hear,parti,hot,noth,late,end,actual,glad,birthday,omg,happen,wow,also,pic,stop,sun,later,weather,wonder,@,put,mom,stuff,ta,saw,stay,fuck,hard,said,run,ya,car,exam,god,world,yesterday,kid,meet,music,that,job,beauti,updat,sunday,ye,mani,friday,post,seem,monday,n,hi,video,found,luck,cold,move,book,die,busi,gone,cri,poor,buy,boy,anyth,least,plan,shop,famili,woke,total,studi,hair,aww,food,almost,cute,iphon,lunch,believ,far,pictur,drink,free,chang,month,place,everyth,sweet,dinner,welcom,funni,tho,win,class,gt,anyon,shit,forward,turn,till,drive,sit,mine,ask,r,okay,walk,name,idea,dream,dad,caus,stupid,write,send,clean,real,coffe,enough,hahaha,ill,room,wrong,probabl,wake,didnt,dog,saturday,anymor,fan,money,minut,person,sooo,hit,ha,rememb,tv,xx,break,aw,headach,came,whole,brother,rock,seen,face,fail,repli,train,beach,hang,eye,kill,blog,=,rest,open,kinda,crazi,pain,close,mother,took,comput,care,word,quit,hell,super,worri,news,true,abl,anyway,hello,forgot,goodnight,els,problem,trip,part,bring,heart,offic,kind,photo,pay,full,link,soo,sister,mind,danc,cuz,b,boo,alon,stuck,internet,fall,test,ticket,cours,sometim,heard,pick,ugh,email,ah,site,awww,set,learn,hehe,pass,vote,interest,text,concert,wont,shower,btw,season,dude,add,awak,fine,fix,],told,ago,hand,favorit,suppos,visit,breakfast,goe,/,cat,sunni,onlin,ice,catch,serious,broke,facebook,bought,spend,ass,pack,cut,wear,lucki,crap,half,asleep,smile,reason,june,ride,lmao,hungri,agre,definit,sign,afternoon,mad,red,instead,ladi,jealou,sore,citi,stori,w/,page,figur,second,join,messag,graduat,album,top,nap,homework,soooo,xd,la,list,congrat,date,togeth,tour,bye,coupl,laptop,dead,park,holiday,wed,sing,save,high,til,water,goin,store,award,+,revis,yea,drop,point,star,moment,complet,relax,dear,perfect,dress,download,church,ppl,short,account,share,side,cook,ipod,line,offici,hug,answer,tea,decid,realiz,youtub,ive,town,annoy,nite,forget,lose,order,scare,weird,gym,mood,unfortun,lil,cream,min,outsid,math,understand,air,english,mum,differ,fb,fli,usual,ate,less,chocol,pool,knew,episod,band,comment,worst,[,upload,fast,support,window,chanc,horribl,flight,load,kick,parent,around,london,question,broken,da,throat,cheer,laugh,team,black,chat,card,sunshin,special,xxx,%,sat,sent,upset,gave,followfriday,sleepi,number,slow,depress,beat,ahh,ad,green,colleg,yep,mac,leg,bless,record,fell,moon,project,appar,paper,what,jona,cake,tuesday,garden,beer,wors,rather,vacat,power,film,websit,doesnt,possibl,longer,warm,app,easi,bet,bodi,bday,blue,due,fair,p,mess,huge,light,c,spent,freak,shoe,finger,hmm,celebr,disappoint,safe,past,lay,juli,cancel,sim,chill,miley,mtv,shame,cousin,bu,earlier,flu,googl,burn,lazi,absolut,age,voic,thx,stomach,white,remind,nope,hold,forev,swim,bike,thursday,manag,stress,wtf,touch,camp,especi,met,shot,camera,appreci,babe,slept,current,sort,son,crash,bum,boyfriend,lie,uk,pray,david,appl,sadli,cd,idk,box,shoot,dm,fight,piss,havent,tom,ahhh,airport,co,type,cover,myspac,block,case,terribl,arriv,present,servic,meant,si,invit,note,expect,bbq,road,pizza,club,bitch,hospit,father,chicken,raini,small,deal,yummi,interview,confus,shirt,tummi,bro,luv,count,fit,tast,cloth,doct

In [34]:
# split X and y into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_state)

In [35]:
# Define all the classifiers to potentially use

et_clf = ExtraTreesClassifier(random_state=random_state)

lda_clf = LinearDiscriminantAnalysis()

gnb_clf = GaussianNB()

cb_clf = CatBoostClassifier(n_estimators = 200,
                           max_depth = None,
                           learning_rate = 0.3,
                           random_state=random_state,
                           cat_features = None,
                           verbose = False)

rf_clf = RandomForestClassifier(random_state = random_state)

mnb_clf = MultinomialNB()

bnb_clf = BernoulliNB()

lr_clf = LogisticRegression(random_state = random_state)

SGD_clf = SGDClassifier(random_state = random_state)



In [36]:
# stack is the actual classifiers being used
# meta_clf is the final one they all feed into
# trial and error in working out the best stack

if stack_to_load == 0: # 1 to load, 0 not to load
    print('training the stack')
    #stack = [rf_clf, et_clf, knn_clf, mnb_clf, bnb_clf, lr_clf, SGD_clf]
    stack = [mnb_clf, bnb_clf, gnb_clf]
    meta_clf = lr_clf

    stack = StackingCVClassifier(classifiers=stack,
                                 use_probas=True,
                                 meta_classifier=meta_clf,
                                 verbose=2,
                                 random_state=random_state)

elif stack_to_load == 1: # 1 to load, 0 not to load
    print('using a pickled version')
    stack_f = open('stack_saved','rb')
    stack = pickle.load(stack_f)
    stack_f.close()
    

training the stack


In [37]:
# training the models
if stack_to_load == 0: # 1 to save, 0 not to save
    print('training the stack')
    stack = stack.fit(X_train, y_train)
else:
    print('not training, using an uploaded pickle')

training the stack
Fitting 3 classifiers...
Fitting classifier1: multinomialnb (1/3)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   15.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting classifier2: bernoullinb (2/3)
BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   21.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   39.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting classifier3: gaussiannb (3/3)
GaussianNB(priors=None, var_smoothing=1e-09)


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   40.0s finished


In [38]:
# save the a prior version of stack for fast loading

if stack_to_save == 1: # 1 to save, 0 not to save
    print('saving a pickled version of the stack')
    stack_Saved = open('stack_Saved','wb')
    pickle.dump(stack,stack_Saved)
    stack_Saved.close()
else:
    print('not saving a pickled version of the stack')


saving a pickled version of the stack


In [39]:
# test the model

prediction = stack.predict(X_test)

print(metrics.accuracy_score(y_test, prediction))

0.7372575


In [40]:
# see ing the accuracy of each classifier stand alone

In [42]:
stack_all = [mnb_clf, bnb_clf, lr_clf, SGD_clf, rf_clf, lda_clf, et_clf, cb_clf]

for i in range(len(stack_all)):
    print(stack_all[i],stack_all[i].fit(X_train, y_train).score(X_test, y_test))

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True) 0.7311575
BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True) 0.73764
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False) 0.7451475
SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=0, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False) 0.74076
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                 

In [43]:
X_train.head(10)

,n't,go,'m,'s,get,day,good,work,love,quot,got,today,time,thank,want,back,miss,lol,know,u,see,feel,think,realli,im,amp,hope,night,watch,still,make,need,new,well,home,look,come,much,last,morn,twitter,tomorrow,wish,great,wait,'re,sleep,haha,sad,fun,tri,right,week,happi,follow,bad,'ve,thing,sorri,tonight,friend,say,way,take,gon,nice,better,hate,even,bed,start,tweet,peopl,school,show,hour,guy,play,weekend,final,hey,lt,let,awesom,use,dont,never,yeah,cant,soon,next,long,littl,rain,pleas,tire,first,everyon,best,movi,year,sick,life,wan,find,girl,sure,call,suck,..,done,help,head,alway,bore,talk,keep,like,alreadi,cool,lot,someth,live,eat,phone,leav,readi,hurt,read,made,man,oh,enjoy,x,went,ok,song,hous,yet,yay,ur,sound,thought,pretti,ever,mayb,na,excit,finish,away,summer,amaz,game,guess,tell,old,mean,one,listen,someon,earli,damn,lost,left,give,check,bit,babi,big,hear,parti,hot,noth,late,end,actual,glad,birthday,omg,happen,wow,also,pic,stop,sun,later,weather,wonder,@,put,mom,stuff,ta,saw,stay,fuck,hard,said,run,ya,car,exam,god,world,yesterday,kid,meet,music,that,job,beauti,updat,sunday,ye,mani,friday,post,seem,monday,n,hi,video,found,luck,cold,move,book,die,busi,gone,cri,poor,buy,boy,anyth,least,plan,shop,famili,woke,total,studi,hair,aww,food,almost,cute,iphon,lunch,believ,far,pictur,drink,free,chang,month,place,everyth,sweet,dinner,welcom,funni,tho,win,class,gt,anyon,shit,forward,turn,till,drive,sit,mine,ask,r,okay,walk,name,idea,dream,dad,caus,stupid,write,send,clean,real,coffe,enough,hahaha,ill,room,wrong,probabl,wake,didnt,dog,saturday,anymor,fan,money,minut,person,sooo,hit,ha,rememb,tv,xx,break,aw,headach,came,whole,brother,rock,seen,face,fail,repli,train,beach,hang,eye,kill,blog,=,rest,open,kinda,crazi,pain,close,mother,took,comput,care,word,quit,hell,super,worri,news,true,abl,anyway,hello,forgot,goodnight,els,problem,trip,part,bring,heart,offic,kind,photo,pay,full,link,soo,sister,mind,danc,cuz,b,boo,alon,stuck,internet,fall,test,ticket,cours,sometim,heard,pick,ugh,email,ah,site,awww,set,learn,hehe,pass,vote,interest,text,concert,wont,shower,btw,season,dude,add,awak,fine,fix,],told,ago,hand,favorit,suppos,visit,breakfast,goe,/,cat,sunni,onlin,ice,catch,serious,broke,facebook,bought,spend,ass,pack,cut,wear,lucki,crap,half,asleep,smile,reason,june,ride,lmao,hungri,agre,definit,sign,afternoon,mad,red,instead,ladi,jealou,sore,citi,stori,w/,page,figur,second,join,messag,graduat,album,top,nap,homework,soooo,xd,la,list,congrat,date,togeth,tour,bye,coupl,laptop,dead,park,holiday,wed,sing,save,high,til,water,goin,store,award,+,revis,yea,drop,point,star,moment,complet,relax,dear,perfect,dress,download,church,ppl,short,account,share,side,cook,ipod,line,offici,hug,answer,tea,decid,realiz,youtub,ive,town,annoy,nite,forget,lose,order,scare,weird,gym,mood,unfortun,lil,cream,min,outsid,math,understand,air,english,mum,differ,fb,fli,usual,ate,less,chocol,pool,knew,episod,band,comment,worst,[,upload,fast,support,window,chanc,horribl,flight,load,kick,parent,around,london,question,broken,da,throat,cheer,laugh,team,black,chat,card,sunshin,special,xxx,%,sat,sent,upset,gave,followfriday,sleepi,number,slow,depress,beat,ahh,ad,green,colleg,yep,mac,leg,bless,record,fell,moon,project,appar,paper,what,jona,cake,tuesday,garden,beer,wors,rather,vacat,power,film,websit,doesnt,possibl,longer,warm,app,easi,bet,bodi,bday,blue,due,fair,p,mess,huge,light,c,spent,freak,shoe,finger,hmm,celebr,disappoint,safe,past,lay,juli,cancel,sim,chill,miley,mtv,shame,cousin,bu,earlier,flu,googl,burn,lazi,absolut,age,voic,thx,stomach,white,remind,nope,hold,forev,swim,bike,thursday,manag,stress,wtf,touch,camp,especi,met,shot,camera,appreci,babe,slept,current,sort,son,crash,bum,boyfriend,lie,uk,pray,david,appl,sadli,cd,idk,box,shoot,dm,fight,piss,havent,tom,ahhh,airport,co,type,cover,myspac,block,case,terribl,arriv,present,servic,meant,si,invit,note,expect,bbq,road,pizza,club,bitch,hospit,father,chicken,raini,small,deal,yummi,interview,confus,shirt,tummi,bro,luv,count,fit,tast,cloth,doct

In [44]:
df_train.head(10)

,target__,id__,date__,flag__,user__,text__,text_cleaned,text_adjectives,text_filtered_stemmed,n't,go,'m,'s,get,day,good,work,love,quot,got,today,time,thank,want,back,miss,lol,know,u,see,feel,think,realli,im,amp,hope,night,watch,still,make,need,new,well,home,look,come,much,last,morn,twitter,tomorrow,wish,great,wait,'re,sleep,haha,sad,fun,tri,right,week,happi,follow,bad,'ve,thing,sorri,tonight,friend,say,way,take,gon,nice,better,hate,even,bed,start,tweet,peopl,school,show,hour,guy,play,weekend,final,hey,lt,let,awesom,use,dont,never,yeah,cant,soon,next,long,littl,rain,pleas,tire,first,everyon,best,movi,year,sick,life,wan,find,girl,sure,call,suck,..,done,help,head,alway,bore,talk,keep,like,alreadi,cool,lot,someth,live,eat,phone,leav,readi,hurt,read,made,man,oh,enjoy,x,went,ok,song,hous,yet,yay,ur,sound,thought,pretti,ever,mayb,na,excit,finish,away,summer,amaz,game,guess,tell,old,mean,one,listen,someon,earli,damn,lost,left,give,check,bit,babi,big,hear,parti,hot,noth,late,end,actual,glad,birthday,omg,happen,wow,also,pic,stop,sun,later,weather,wonder,@,put,mom,stuff,ta,saw,stay,fuck,hard,said,run,ya,car,exam,god,world,yesterday,kid,meet,music,that,job,beauti,updat,sunday,ye,mani,friday,post,seem,monday,n,hi,video,found,luck,cold,move,book,die,busi,gone,cri,poor,buy,boy,anyth,least,plan,shop,famili,woke,total,studi,hair,aww,food,almost,cute,iphon,lunch,believ,far,pictur,drink,free,chang,month,place,everyth,sweet,dinner,welcom,funni,tho,win,class,gt,anyon,shit,forward,turn,till,drive,sit,mine,ask,r,okay,walk,name,idea,dream,dad,caus,stupid,write,send,clean,real,coffe,enough,hahaha,ill,room,wrong,probabl,wake,didnt,dog,saturday,anymor,fan,money,minut,person,sooo,hit,ha,rememb,tv,xx,break,aw,headach,came,whole,brother,rock,seen,face,fail,repli,train,beach,hang,eye,kill,blog,=,rest,open,kinda,crazi,pain,close,mother,took,comput,care,word,quit,hell,super,worri,news,true,abl,anyway,hello,forgot,goodnight,els,problem,trip,part,bring,heart,offic,kind,photo,pay,full,link,soo,sister,mind,danc,cuz,b,boo,alon,stuck,internet,fall,test,ticket,cours,sometim,heard,pick,ugh,email,ah,site,awww,set,learn,hehe,pass,vote,interest,text,concert,wont,shower,btw,season,dude,add,awak,fine,fix,],told,ago,hand,favorit,suppos,visit,breakfast,goe,/,cat,sunni,onlin,ice,catch,serious,broke,facebook,bought,spend,ass,pack,cut,wear,lucki,crap,half,asleep,smile,reason,june,ride,lmao,hungri,agre,definit,sign,afternoon,mad,red,instead,ladi,jealou,sore,citi,stori,w/,page,figur,second,join,messag,graduat,album,top,nap,homework,soooo,xd,la,list,congrat,date,togeth,tour,bye,coupl,laptop,dead,park,holiday,wed,sing,save,high,til,water,goin,store,award,+,revis,yea,drop,point,star,moment,complet,relax,dear,perfect,dress,download,church,ppl,short,account,share,side,cook,ipod,line,offici,hug,answer,tea,decid,realiz,youtub,ive,town,annoy,nite,forget,lose,order,scare,weird,gym,mood,unfortun,lil,cream,min,outsid,math,understand,air,english,mum,differ,fb,fli,usual,ate,less,chocol,pool,knew,episod,band,comment,worst,[,upload,fast,support,window,chanc,horribl,flight,load,kick,parent,around,london,question,broken,da,throat,cheer,laugh,team,black,chat,card,sunshin,special,xxx,%,sat,sent,upset,gave,followfriday,sleepi,number,slow,depress,beat,ahh,ad,green,colleg,yep,mac,leg,bless,record,fell,moon,project,appar,paper,what,jona,cake,tuesday,garden,beer,wors,rather,vacat,power,film,websit,doesnt,possibl,longer,warm,app,easi,bet,bodi,bday,blue,due,fair,p,mess,huge,light,c,spent,freak,shoe,finger,hmm,celebr,disappoint,safe,past,lay,juli,cancel,sim,chill,miley,mtv,shame,cousin,bu,earlier,flu,googl,burn,lazi,absolut,age,voic,thx,stomach,white,remind,nope,hold,forev,swim,bike,thursday,manag,stress,wtf,touch,camp,especi,met,shot,camera,appreci,babe,slept,current,sort,son,crash,bum,boyfriend,lie,uk,pray,david,appl,sadli,cd,idk,box,shoot,dm,fight,piss,havent,tom,ahhh,airport,co,type,cover,myspac,block,case,terribl,arriv,present,servic,meant,si,invit,note,expect,bbq,road,pizza,club,bitch,hospit,father,